In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import imageio.v3 as imageio
from tqdm.auto import trange
import texture_segmentation.gabor as gabor
import texture_segmentation.diffusion as diffusion

plt.rcParams['image.cmap'] = 'gray'

In [ ]:
imsize = 300
# backgroud_texture_id = 9
# inner_texture_id = 80
backgroud_texture_id = 16
inner_texture_id = 77

bg_image_url = f"https://www.ux.uis.no/~tranden/brodatz/D{backgroud_texture_id}.gif"
texture_image_url = f"https://www.ux.uis.no/~tranden/brodatz/D{inner_texture_id}.gif"


bg_image = imageio.imread(bg_image_url)
bg_image = bg_image[:, 0:imsize, 0:imsize].astype(float) / 255

texture_image = imageio.imread(texture_image_url)
texture_image = texture_image[:, 0:imsize, 0:imsize].astype(float) / 255
mask = np.zeros_like(bg_image)
mask[:, imsize//3:-imsize//3, imsize//3:-imsize//3] = 1
mask = sp.ndimage.gaussian_filter(mask, sigma=(0, 5, 5))

texture_image = texture_image - texture_image.mean() + bg_image.mean()
texture_image = texture_image * bg_image.std() / texture_image.std()
texture_image = np.clip(texture_image, 0, 1)

image = (1 - mask) * bg_image + mask * texture_image
image -= image.min()
image /= image.max()
# image = np.random.normal(image, 0.1)
image = np.clip(image, 0, 1)
image = image[0]
plt.imshow(image)
plt.show()

In [ ]:
filter_bank_params = {
    'num_angles': 20,
    'full_circle': False,
    'scaling_factor': 0.4,
    'num_scales': 4,
    'sigma_x0': 0.17,
    "sigma_y0": 0.07
}

arr = gabor.plot_gabor_filter_bank_fft_fwhm(300, **filter_bank_params)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_aspect('equal')
ax.imshow(arr, cmap="binary")
ax.set_axis_off()
plt.tight_layout()
plt.show()

In [ ]:
gabor_features_complex = gabor.gabor_features_raw(image, filter_bank_params)

# im = np.zeros((*gabor_features_complex.shape, 3), dtype=float)
# im[..., 0] = gabor_features_complex.real
# im[..., 2] = gabor_features_complex.imag

im = np.abs(gabor_features_complex)
vmin = np.quantile(im, 0.01)
vmax = np.quantile(im, 0.99)
# vmin = im.min()
# vmax = im.max()
im = im - vmin
im = im / vmax


# Plot the complex gabor features
s = 5
fig, axs = plt.subplots(nrows=im.shape[0], ncols=im.shape[1], figsize=(int(s * im.shape[1] / im.shape[0]), s))
for i in range(im.shape[0]):
    for j in range(im.shape[1]):
        axs[i, j].imshow(im[i, j], vmin=0, vmax=1)
        axs[i, j].axis("off")
plt.tight_layout()
plt.show()

In [ ]:
gabor_resoponse = np.abs(gabor_features_complex)
n, m = gabor_resoponse.shape[:2]
gabor_resoponse = gabor_resoponse.reshape(n * m, *gabor_resoponse.shape[2:])
max_idx = np.argmax(gabor_resoponse, axis=0)
# gabor_resoponse = gabor_resoponse.reshape(n, m, *gabor_resoponse.shape[1:])
max_scale_idx, max_angle_idx = np.unravel_index(max_idx, (n, m))
xx, yy = np.mgrid[0:gabor_resoponse.shape[-2], 0:gabor_resoponse.shape[-1]]
print(xx.shape, yy.shape)
max_complex_gabor_response = gabor_features_complex[max_scale_idx, max_angle_idx, xx, yy]
max_angle = max_angle_idx * np.pi / filter_bank_params['num_angles']
max_scale = filter_bank_params['scaling_factor'] ** max_scale_idx

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 10))
axs[0, 0].imshow(max_complex_gabor_response.real)
axs[0, 0].set_title("Real part")
axs[0, 0].axis("off")
axs[0, 1].imshow(max_complex_gabor_response.imag)
axs[0, 1].set_title("Imaginary part")
axs[0, 1].axis("off")
axs[1, 0].imshow(max_angle)
axs[1, 0].set_title("Angle")
axs[1, 0].axis("off")
axs[1, 1].imshow(max_scale)
axs[1, 1].set_title("Scale")
axs[1, 1].axis("off")
plt.tight_layout()

plt.show()

In [ ]:
m, k, h, w = gabor_features_complex.shape
X = np.abs(gabor_features_complex).reshape(-1, *gabor_features_complex.shape[-2:])
means = X.mean(axis=0, keepdims=True)
stds = X.std(axis=0, keepdims=True)
X = (X - means) / stds
X = sp.ndimage.gaussian_filter(X, sigma=(0, 11, 11))
X = diffusion.diffuse_features(X, it=100, eta=0.1)
X = X * stds + means

im_d = np.abs(X).reshape(m, k, h, w)

# im_d = im_d
# vmin = im_d.min()
# vmax = im_d.max()

im_d = im_d - vmin
im_d = im_d / vmax
im_d = np.clip(im_d, 0, 1)


# Plot the complex gabor features
s = 5
fig, axs = plt.subplots(nrows=im_d.shape[0], ncols=im_d.shape[1], figsize=(int(s * im_d.shape[1] / im_d.shape[0]), s))
for i in range(im_d.shape[0]):
    for j in range(im_d.shape[1]):
        axs[i, j].imshow(im_d[i, j], vmin=0, vmax=1)
        axs[i, j].axis("off")
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.decomposition import PCA
import math

pixels = X.reshape(X.shape[0], -1).T
pca = PCA(n_components=10)
pca.fit(pixels)

pixels_pca = pca.transform(pixels).T
pixels_pca = pixels_pca.reshape(10, h, w)
plt.imshow(pixels_pca[0])
plt.show()


In [ ]:
grad = np.gradient(pixels_pca[0])
grad = np.sqrt(grad[0]**2 + grad[1]**2)
im = 1 / (1 + grad ** 2)

vmin = np.quantile(im, 0.05)
vmax = np.quantile(im, 0.95)
plt.imshow(im , vmin=vmin, vmax=vmax)
plt.show()